# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import time
import nltk
nltk.download('words')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\srini\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\srini\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\srini\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\srini\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\srini\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\srini\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, f1_score, accuracy_score, precision_score, recall_score, make_scorer
import pickle
from IPython.display import FileLink
from sklearn.svm import SVC

In [3]:
# load data from database
engine = create_engine('sqlite:///DisasterDB.db')
df = pd.read_sql_table('EmergencyMessage', engine, )
df.head(2)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0


In [4]:
X= df.message
Y= df.iloc[:,4:]

In [5]:
# Dropping the column with no 1 lables
Y.drop(columns='child_alone', inplace= True)

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    """Function to tokenize the given text.
    It removes punctuation, lowers the case, removes the stopwords, lemmatizes and stems the words in the text.
    
    Parameters:
    text: str
        The input text that needs to be tokenized
    
    Returns:
    List that is tokenized """
    
    # replace punctuations with spaces and change text to lower case
    temp = word_tokenize(re.sub(r'[\W]',' ',text).lower())
    
    # remove stop words from the sentence
    words= [x for x in temp if x not in stopwords.words('english')]
    
    # lemmatize and stem the words
    return [PorterStemmer().stem( WordNetLemmatizer().lemmatize(w)) for w in words]

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline(
[ ('vect', CountVectorizer(tokenizer= tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size= 0.3 )
X_train.shape, y_train.shape

((18350,), (18350, 35))

In [9]:
pipeline.fit(X_train, y_train)

C:\Users\srini\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\srini\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\srini\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\srini\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.2

C:\Users\srini\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\srini\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                  

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
y_train_pred= pipeline.predict(X_train)

In [11]:
y_test_pred= pipeline.predict(X_test)

In [12]:
f1_test_score= []

for i,col in enumerate(y_test.columns):
    print(classification_report(y_test[col], np.transpose(y_test_pred)[i]))

              precision    recall  f1-score   support

           0       0.61      0.45      0.52      1825
           1       0.85      0.91      0.88      6040

    accuracy                           0.81      7865
   macro avg       0.73      0.68      0.70      7865
weighted avg       0.79      0.81      0.79      7865

              precision    recall  f1-score   support

           0       0.90      0.97      0.93      6516
           1       0.78      0.45      0.57      1349

    accuracy                           0.88      7865
   macro avg       0.84      0.71      0.75      7865
weighted avg       0.88      0.88      0.87      7865

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7827
           1       0.00      0.00      0.00        38

    accuracy                           1.00      7865
   macro avg       0.50      0.50      0.50      7865
weighted avg       0.99      1.00      0.99      7865

              preci

C:\Users\srini\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\srini\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [81]:
# weighted average f1 score
f1_score(np.array(y_test), y_test_pred, average='micro')

0.6186026704075753

### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
# Setting up a grid search with Random forest to find the best parameters
parameters = {'clf__estimator__max_depth': [300], #, 200, 250 
             #'clf__estimator__min_samples_leaf': [1,4],
             #'clf__estimator__min_samples_split': [2,5],
              'clf__estimator__n_estimators': [80], # [20,80]
              #'tfidf__use_idf':[True, False]
             }

my_scorer = make_scorer(f1_score,average='micro' )

cv = GridSearchCV(estimator= pipeline, param_grid= parameters, scoring= my_scorer, cv=3, verbose= 3, n_jobs= 2 )

In [12]:
cv.fit(X_train, y_train )

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   3 out of   3 | elapsed:  8.4min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                            

In [103]:
cv.best_params_

{'clf__estimator__max_depth': 300,
 'clf__estimator__min_samples_leaf': 1,
 'clf__estimator__min_samples_split': 5,
 'clf__estimator__n_estimators': 80}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [13]:
def get_train_test_score(X_train, X_test, y_train, y_test, grid_search=False):
    """ Function that returns the average F1 score, precision, accuracy and recall for given training and testing set.
    The response (y_train/y_test) is a multi-column/ category output.
    Input: Training data
    Output: Average F1, accuracy, precision, recall scores across all the columns"""
    
    # Calculating the predicted values from the model
    if grid_search==True:
        y_train_pred= cv.predict(X_train)
        y_test_pred= cv.predict(X_test)
    if grid_search== False:
        y_train_pred= pipeline.predict(X_train)
        y_test_pred= pipeline.predict(X_test)
    
    print('F1 \nTrain:', f1_score(y_train,y_train_pred, average= 'micro')  )
    print('F1 \nTest:', f1_score(y_test,y_test_pred, average= 'micro'),  )

get_train_test_score(X_train, X_test, y_train, y_test, grid_search= True)

F1 
Train: 0.9819456800460516
F1 
Test: 0.6519033145172962


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [107]:
#Using SVC algorithm for classification
pipeline2 = Pipeline(
[ ('vect', CountVectorizer(tokenizer= tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(SVC() ))
])

In [108]:
pipeline2.fit(X_train, y_train)

C:\Users\srini\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\srini\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\srini\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\User

C:\Users\srini\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\srini\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\srini\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\User

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultiOutputClassifier(estimator=SVC(C=1.0, cache_size=200,
                                                     class_weight=None,
                               

In [ ]:
parameter2= {'clf__estimator__C': [10,100,1000],
    'clf__estimator__kernel': ['rbf','poly'],
    #'clf__estimator__degree': [3,4],
    'clf__estimator__gamma':['scale']
}
my_scorer = make_scorer(f1_score,average='micro' )

cv2 = GridSearchCV(estimator= pipeline2, param_grid= parameter2, scoring= my_scorer, cv=3, verbose= 3, n_jobs= 2 )

cv2.fit(X_train, y_train)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


In [ ]:

y_train_pred= cv2.predict(X_train)
y_test_pred= cv2.predict(X_test)
    
print('F1 \nTrain:', f1_score(y_train,y_train_pred, average= 'micro')  )
print('F1 \nTest:', f1_score(y_test,y_test_pred, average= 'micro'),  )

In [110]:
pipeline2.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x00000186C1572828>,
                   vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=SVC(C=1.0, cache_size=200, class_weight=None,
                                       coef0=0.0, decision_function_shape='ovr',
                                       degree=3, gamma='auto_deprecated',
                                       kernel='rbf', max_iter=-1,
                                       probability=False, random

### 9. Export your model as a pickle file

In [66]:
pickle.dump(cv, open('random_forest_model.pkl','wb'))
FileLink(r'random_forest_model.pkl')

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.